In [1]:
import json
from io import open

file_path = "2020_acl_diplomacy/data/train.jsonl"
train_data = {}

with open(file_path, 'r') as file:
    i = 0
    for line in file:
        # Parse the line as JSON
        train_data[i] = json.loads(line)
        i+=1




In [2]:
file_path = "2020_acl_diplomacy/data/test.jsonl"
test_data = {}

with open(file_path, 'r') as file:
    i = 0
    for line in file:
        # Parse the line as JSON
        test_data[i] = json.loads(line)
        i+=1

In [3]:
# print(len(diplomacy_data))
# for key, value in diplomacy_data[27].items():
#     print("Key: ", key)
messages_26= train_data[36]["messages"]
sender_26= train_data[36]["sender_labels"]
receiever_26= train_data[36]["receiver_labels"]

# print(len(messages_26), len(sender_26), len(receiever_26))

i=0
for message in messages_26:
    print(message, sender_26[i], receiever_26[i])
    i+=1


Hello Russia. Have you worked out what you’re doing with turkey and Austria yet? I’m thinking of going anti Germany but want to see how we align in the north first. False NOANNOTATION
I sent Turkey a message but still waiting on a reply. I wanted to go anti-Germany too if you want to split the population centers? But if I don’t hear back from Turkey I may have to fortify my South flank and enter the Balkan’s even though I don’t want to. True NOANNOTATION
I moving to pressureGermany but I could use assistance on your end. Willing to share any gains that could result. True True
Yikes. You have some Turkish trouble. I’ll plan on moving into helgoland bight True NOANNOTATION
Yea, that’s always annoying playing as Russia. Trying to get AH to help out but I have to move one of my troops back West away from pressuring Germany. Oh well. True True
That’s a shame. Let’s reassess after you handle your problems in the east. True NOANNOTATION
Hey man. Let me know if you wanna support my fleet movin

In [4]:
%pip install transformers
# from transformers import T5Tokenizer


inputs, targets = [], []

for key, instance in train_data.items():
    if isinstance(instance, dict) and 'messages' in instance:
        messages = instance['messages']
        sender_labels = instance['sender_labels']

    for idx, (message, label) in enumerate(zip(messages, sender_labels)):
        start_index = max(0, idx-10)
        end_index = min(len(messages)-1, idx)
        context = " ".join(messages[start_index:end_index])
        inputs.append(f"context: {context} classify: {message}")
        targets.append("true" if label else "false")


print(len(inputs), len(targets)) 
print(inputs[3])
print(targets[3])

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3.9 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
13132 13132
context: Germany!

Just the person I want to speak with. I have a somewhat crazy idea that I’ve always wanted to try with I/G, but I’ve never actually convinced the other guy to try it. And, what’s worse, it might make you suspicious of me. 

So...do I suggest it?

I’m thinki

In [5]:
test_inputs, test_targets = [], []

for key, instance in test_data.items():
    if isinstance(instance, dict) and 'messages' in instance:
        messages = instance['messages']
        sender_labels = instance['sender_labels']

    for idx, (message, label) in enumerate(zip(messages, sender_labels)):
        start_index = max(0, idx-10)
        end_index = min(len(messages)-1, idx)
        context = " ".join(messages[start_index:end_index])
        test_inputs.append(f"context: {context} classify: {message}")
        test_targets.append("true" if label else "false")


In [6]:
%pip install sentencepiece
# tokenizer = T5Tokenizer.from_pretrained('t5-small')

# train_encodings = tokenizer(inputs, truncation=True, padding='max_length', max_length=512)
# train_labels = tokenizer(targets, truncation=True, padding='max_length', max_length=512)

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3.9 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install torch
import torch
from torch.utils.data import Dataset

class DiplomacyDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

        assert len(self.encodings['input_ids']) == len(self.labels['input_ids']), "Encodings and labels must have the same number of samples."


    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels['input_ids'][idx])
        return item

    def __len__(self):
        return len(self.labels)

# train_dataset = DiplomacyDataset(train_encodings, train_labels)


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3.9 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
%pip install imbalanced-learn

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3.9 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:

file_path = "2020_acl_diplomacy/data/validation.jsonl"
validation_data = {}

with open(file_path, 'r') as file:
    i = 0
    for line in file:
        # Parse the line as JSON
        validation_data[i] = json.loads(line)
        i+=1




In [10]:
val_texts, targetval_texts = [], []

for key, instance in validation_data.items():
    if isinstance(instance, dict) and 'messages' in instance:
        messages = instance['messages']
        sender_labels = instance['sender_labels']

    for idx, (message, label) in enumerate(zip(messages, sender_labels)):
        start_index = max(0, idx-10)
        end_index = min(len(messages)-1, idx)
        context = " ".join(messages[start_index:end_index])
        val_texts.append(f"context: {context} classify: {message}")
        targetval_texts.append("true" if label else "false")

In [11]:
# %pip install allennlp
from transformers import BertModel, BertTokenizer
from transformers import AdamW
from sklearn.metrics import f1_score, accuracy_score
import numpy as np
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler




model = BertModel.from_pretrained('bert-base-uncased')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
for param in model.parameters():
    param.requires_grad = False
val_encodings = tokenizer(val_texts, return_tensors="pt", padding='max_length', max_length=512, truncation=True)
# val_labels = tokenizer(targetval_texts, truncation=True, padding='max_length', max_length=512)

# Pass inputs to the model
# x_outputs = model(**inputs)
x_embeddings = []

print(len(inputs))
for text in inputs:
    tokens = tokenizer(text, return_tensors="pt", padding='max_length', max_length=512, truncation=True)
    with torch.no_grad():
        outputs = model(**tokens)
    # print(outputs.pooler_output[0].numpy().shape)
    x_embeddings.append(outputs.pooler_output.squeeze().numpy())

smote = SMOTE(sampling_strategy=0.5, k_neighbors=3, random_state=42)

X = np.stack(x_embeddings)
Y = np.array(targets)  
X_resampled, Y_resampled = smote.fit_resample(X, Y)
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X_resampled)
clf = LogisticRegression(class_weight='balanced', solver='liblinear')

clf.fit(X_resampled, Y_resampled)


print("Shape of X:", X_resampled.shape)
print("Shape of Y:", Y_resampled.shape)
# clf.fit(X_resampled, Y_resampled)


/opt/homebrew/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


13132
Shape of X: (17557, 768)
Shape of Y: (17557,)


In [12]:
x_test_embeddings = []

print(len(test_inputs))
for text in test_inputs:
    tokens = tokenizer(text, return_tensors="pt", padding='max_length', max_length=512, truncation=True)
    with torch.no_grad():
        outputs = model(**tokens)
    # print(outputs.pooler_output[0].numpy().shape)
    x_test_embeddings.append(outputs.pooler_output.squeeze().numpy())


X_test = np.stack(x_test_embeddings)
Y_test = np.array(test_targets)  

y_pred = clf.predict(X_test)

accuracy = accuracy_score(Y_test, y_pred)
print("Accuracy:", accuracy)



2741
Accuracy: 0.7526450200656695


In [13]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(Y_test, y_pred))

# Confusion matrix
cm = confusion_matrix(Y_test, y_pred)
print("Confusion Matrix:\n", cm)

              precision    recall  f1-score   support

       false       0.13      0.32      0.19       240
        true       0.92      0.79      0.85      2501

    accuracy                           0.75      2741
   macro avg       0.53      0.56      0.52      2741
weighted avg       0.85      0.75      0.80      2741

Confusion Matrix:
 [[  77  163]
 [ 515 1986]]


In [14]:

# optimizer = AdamW(model.parameters())

# class MyTrainer(Trainer):
#     def create_optimizer(self):
#         if not hasattr(self, "optimizer"):
#             self.optimizer = AdamW(self.model.parameters(), lr=self.args.learning_rate)
#         return self.optimizer

# training_args = TrainingArguments(
#     output_dir='./results',
#     per_device_train_batch_size=8,
#     num_train_epochs=1,
#     logging_dir='./logs',
#     logging_steps=100,
#     save_steps=1000,
#     save_total_limit=3
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#     tokenizer=tokenizer,
# )

# trainer.create_optimizer

# for i in range(30):
#     trainer.train()
#     results = trainer.evaluate()
#     print(results)
#     prediction_outputs = trainer.predict(val_dataset)
#     predicted_labels = np.argmax(prediction_outputs.predictions[0], axis=2)
#     predicted_labels = predicted_labels.flatten()
#     label_ids = prediction_outputs.label_ids.flatten()

#     # y_pred = np.around(model.predict(val_dataset))

#     print(metrics.classification_report(label_ids, predicted_labels))

#     # f1 = f1_score(label_ids, predicted_labels, average="weighted")
    # print(f"F1 Score: {f1}")

    # acc = accuracy_score(label_ids, predicted_labels, normalize=True)
    # print(f"Accuracy: {acc}")

# print accuracy and loss at each step
# what is the input and what is expected output for both classification and generation